In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from collections import deque
import numpy as np

import time
import random
import tensorflow as tf
import os
from tqdm import tqdm
from PIL import Image
import cv2
import keras

from blob import Blob, BlobEnv

DISCOUNT = 0.99
REPLAY_MEMORY_SIZE = 50_000  # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 1_000  # Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 64  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)
MODEL_NAME = '2x256'
MIN_REWARD = -200  # For model save
MEMORY_FRACTION = 0.20

# Environment settings
EPISODES = 10

# Exploration settings
epsilon = 0  # not a constant, going to be decayed
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001

#  Stats settings
AGGREGATE_STATS_EVERY = 5  # episodes
SHOW_PREVIEW = True

In [32]:
# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        
        # OLD CODE
        # AttributeError: module 'tensorboard.summary._tf.summary' has no attribute 'FileWriter' 
        # self.writer = tf.summary.FileWriter(self.log_dir)
        
        # FROM GITHUB FIX
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self._log_write_dir = self.log_dir


    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        
        # self.model = model
        # self._train_dir = os.path.join(self._log_write_dir, 'train')
        # self._train_step = self.model._train_counter
        # self._val_dir = os.path.join(self._log_write_dir, 'validation')
        # self._val_step = self.model._test_counter
        # self._should_write_train_graph = False
          
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        pass
        # self._write_logs(stats, self.step)      
        # with self.writer.as_default():
        #     for key, value in stats.items():
        #         tf.summary.scalar(key, value, step = self.step)
        #         self.writer.flush()
    
    
    
# AttributeError: 'ModifiedTensorBoard' object has no attribute '_write_logs'

    # GITHUB SOLUTION
    def _write_logs(self, logs, index):
        pass
        # with self.writer.as_default():
        #     for name, value in logs.items():
        #         tf.summary.scalar(name, value, step=index)
        #         self.step += 1
        #         self.writer.flush()
                
# AttributeError: 'ModifiedTensorBoard' object has no attribute '_train_dir'

                
                
                
                
# class ModifiedTensorBoard(TensorBoard):

# def __init__(self, **kwargs):
#     super().__init__(**kwargs)
#     self.step = 1
#     self.writer = tf.summary.create_file_writer(self.log_dir)
#     self._log_write_dir = self.log_dir

# def set_model(self, model):
#     self.model = model

#     self._train_dir = os.path.join(self._log_write_dir, 'train')
#     self._train_step = self.model._train_counter

#     self._val_dir = os.path.join(self._log_write_dir, 'validation')
#     self._val_step = self.model._test_counter

#     self._should_write_train_graph = False

# def on_epoch_end(self, epoch, logs=None):
#     self.update_stats(**logs)

# def on_batch_end(self, batch, logs=None):
#     pass

# def on_train_end(self, _):
#     pass

# def update_stats(self, **stats):
#     with self.writer.as_default():
#         for key, value in stats.items():
#             tf.summary.scalar(key, value, step = self.step)
#             self.writer.flush()

# ACTUAL CODE

In [26]:
class DQNAgent:
    def __init__(self):
        # main model, gets trained every step
        self.model = self.create_model()
        
        # used to predict every step
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        
        self.replay_memory = deque(maxlen = REPLAY_MEMORY_SIZE)
        
        self.tensorboard = ModifiedTensorBoard(log_dir = f'logs/{MODEL_NAME}-{int(time.time())}')
        
        self.target_update_counter = 0
        
    def create_model(self):
        model = Sequential()
        model.add(Conv2D(256, (3,3), input_shape = env.OBSERVATION_SPACE_VALUES))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2,2))
        model.add(Dropout(0.2))
        
        model.add(Conv2D(256, (3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(2,2))
        model.add(Dropout(0.2))
        
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Dense(env.ACTION_SPACE_SIZE, activation = 'linear'))
        
        model.compile(loss = 'mse', optimizer = Adam(learning_rate= 0.001), metrics = ['accuracy'])
        
        return model
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/255)[0]
    def train(self, terminal_state, step):
        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # Get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0] for transition in minibatch])/255
        current_qs_list = self.model.predict(current_states)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array([transition[3] for transition in minibatch])/255
        future_qs_list = self.target_model.predict(new_current_states)
        
        
        
        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X)/255, np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False, callbacks=[self.tensorboard] if terminal_state else None)
        
        
        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
            
            


In [27]:
env = BlobEnv()

# For stats
# WHY IS THIS [-200]????
ep_rewards = [-200]

# For more repetitive results
random.seed(1)
np.random.seed(1)
# tf.set_random_seed(1)
tf.random.set_seed(1)

# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')

In [28]:
agent = DQNAgent()

2022-10-24 11:09:32.851958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
agent.model

In [30]:
agent.model = keras.models.load_model("models/2x256____25.00max_-184.30avg_-443.00min__1666598499.model")

In [31]:
agent.model

In [15]:
# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1

    # Reset environment and get initial state
    current_state = env.reset()
    # Reset flag and start iterating until episode ends
    done = False
    while not done:

        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, env.ACTION_SPACE_SIZE)

        new_state, reward, done = env.step(action)
        
        # Transform new continous state to new discrete state and count reward
        episode_reward += reward

        if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
            env.render()

        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)
        current_state = new_state
        step += 1
        
        
        
    # Append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if average_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)


NameError: name 'tqdm' is not defined

In [10]:
1+1

2

In [ ]:
x = deque(maxlen = 3)
x

In [ ]:
x.append(3)

In [ ]:
from functools import reduce
primes = [2, 3, 5, 7, 11, 13]
def product(*numbers):
    print(f'numbers = {numbers}')
    p = reduce(lambda x, y: x * y, numbers)
    return p 
product(*[2,3,4])

In [ ]:
x

In [ ]:
x.append('5')

In [ ]:
x

In [ ]:
x.append((1,2))

In [ ]:
x

In [ ]:
x.append('hi')

In [ ]:
x

In [ ]:
x.append(x)

In [ ]:
x

In [ ]:
x[2][2][2]